In [27]:
import numpy as np
from pyomo.environ import *
from pyomo.gdp import *

In [61]:
CLEANING_TIMES = [
    [0, 11, 7, 13, 11],
    [5, 0, 13, 15, 15],
    [13, 15, 0, 23, 11],
    [9, 13, 5, 0, 3],
    [3, 7, 7, 7, 0]
]

MIXING_TIMES = [40, 35, 45, 32, 50]

TASKS = {
    'batch1': {'cleaning_time': CLEANING_TIMES[0], 'mixing_time': MIXING_TIMES[0]},
    'batch2': {'cleaning_time': CLEANING_TIMES[1], 'mixing_time': MIXING_TIMES[1]},
    'batch3': {'cleaning_time': CLEANING_TIMES[2], 'mixing_time': MIXING_TIMES[2]},
    'batch4': {'cleaning_time': CLEANING_TIMES[3], 'mixing_time': MIXING_TIMES[3]},
    'batch5': {'cleaning_time': CLEANING_TIMES[4], 'mixing_time': MIXING_TIMES[4]}
}

succ = np.array(CLEANING_TIMES).astype(np.bool_)

In [52]:
model = ConcreteModel()
model.TASKS = Set(initialize = TASKS.keys())

AbstractOrderedScalarSet


In [44]:
def define_model(TASKS):
    model = ConcreteModel()

    model.TASKS = Set(initialize = TASKS.keys(), dimen=2)
    model.JOBS = Set(initialize = list(set([j for (j,m) in model.TASKS])))
    model.MACHINES = Set(initialize = list(set([m for (j,m) in model.TASKS])))
    model.TASKORDER = Set(initialize = model.TASKS * model.TASKS, dimen=4,
        filter = lambda _, j, m, k, n: (k,n) == TASKS[(j,m)]['exists'])

    model.dur = Param(model.TASKS, initialize=lambda _, j, m: TASKS[(j,m)]['dur'])
    
    upperBound = sum([model.dur[j, m] for (j,m) in model.TASKS])
    model.makespan = Var(bounds=(0, upperBound))
    model.start = Var(model.TASKS, bounds=(0, upperBound))
    model.succ = Var(model.TASKS, initialize=lambda _, j, m: succ[j][m])
    model.objective = Objective(expr = model.makespan, sense = minimize)
    model.finish = Constraint(model.TASKS, rule=lambda model, j, m:
        (model.start[j,m] + model.dur[j,m]) * model.succ[j,m])

    model.preceding = Constraint(model.TASKORDER, rule=lambda model, j, m, k, n:
        model.start[k,n] + model.dur[k,n] <= model.start[j,m])

    model.disjunctions = Disjunction(model.DISJUNCTIONS, rule=lambda model,j,k,m:
        [model.start[j,m] + model.dur[j,m] <= model.start[k,m],
         model.start[k,m] + model.dur[k,m] <= model.start[j,m]])

    TransformationFactory('gdp.hull').apply_to(model)
    return model